In [1]:
from src.signal_generator import SignalGenerator
from src.metrics import Metric
from src.data_type import Measurement
import matplotlib.pyplot as plt
from src.data_compressor.other import CompressNTHS
from src.data_compressor.other import CompressMinMax
from src.data_compressor.other import CompressPWP
from src.data_compressor.pip import CompressPIP_ED
from src.data_compressor.pip import CompressPIP_PD
from src.data_compressor.pip import CompressPIP_VD
from src.data_compressor.paa import CompressPAA
from src.data_compressor.paa import CompressPAAVI
from src.data_compressor.paa import CompressByChunk
from src.data_compressor.pla import CompressAPCADFT
from src.data_compressor.pla import CompressAPCAFFT
from src.data_compressor.pla import CompressSTC
from src.data_compressor.pla import CompressHigherDeriveration

In [2]:
from src.data_type import Measurement
from src.data_compressor.compressor import Compressor
from typing import List, Dict
from collections import defaultdict as dict

In [3]:
signal_generator = SignalGenerator(0, 100).with_peaks(3).with_peaks(3, direction=-1).sin(0.2, 0.2)
measurements = [Measurement(measurement, index * 100) for index, measurement in enumerate(signal_generator.data)]

In [4]:
class AlgorythmSelector:

  def __init__(self) -> None:
    self.metrics_containter = Metric()
    self.compressors: Dict[str, Compressor] = {
      'CompressNTHS': CompressNTHS(),
      'CompressMinMax': CompressMinMax()
    }

  def set_metrics(self):
    pass # todo
  
  def get_best(self, data: List[Measurement]):
    metric_result = dict()
    for name, compressor in self.compressors.items():
      compressor.set_data(data)
      compressor.compress()
      compressed_data = compressor.compressed_data
      metric_result[name] = {
        'compression_rate': self.metrics_containter.compression_ratio(data, compressed_data),
        'sum_differences': self.metrics_containter.sum_differences(data, compressed_data),
        'arithmetic_average': self.metrics_containter.arithmetic_average(data, compressed_data),
        'standard_derivative': self.metrics_containter.standard_derivative(data, compressed_data),
      }
    for key, metrics in metric_result.items():
      print(key)
      for name, value in metrics.items():
        print(f"\t{name}:\t{value}")
    return list(self.compressors.items())[0][1]

AlgorythmSelector().get_best(measurements)

CompressNTHS
	compression_rate:	0.35
	sum_differences:	2.882062507135529
	arithmetic_average:	0.02882062507135529
	standard_derivative:	0.08634639929437112
CompressMinMax
	compression_rate:	0.5
	sum_differences:	2.2588701404928253
	arithmetic_average:	0.022588701404928253
	standard_derivative:	0.06289878759161126
